## Model Optimization

## Preprocessing

In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_application_df = application_df.drop(columns=['EIN', 'NAME'])
clean_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [31]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = clean_application_df['APPLICATION_TYPE'].value_counts()[clean_application_df['APPLICATION_TYPE'].value_counts() < 500]

# Replace in dataframe
for app in application_types_to_replace.index:
    clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace =  clean_application_df['CLASSIFICATION'].value_counts()[clean_application_df['CLASSIFICATION'].value_counts() < 1000]

# Replace in dataframe
for cls in classifications_to_replace.index:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [33]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_wdummies = pd.get_dummies(clean_application_df)
application_df_wdummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
# Split our preprocessed data into our features and target arrays
y = application_df_wdummies.IS_SUCCESSFUL.values
X = application_df_wdummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [35]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu', input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                1760      
                                                                 
 dense_4 (Dense)             (None, 10)                410       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2181 (8.52 KB)
Trainable params: 2181 (8.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Export our model to HDF5 file
checkpoint = "model_checkpoint.h5"
checkpoint_callback = ModelCheckpoint("model_checkpoint.h5", save_weights_only=True, save_freq=5)

In [38]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback], validation_data=(X_test, y_test))

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.5793 - accuracy: 0.7130 - val_loss: 8563.7754 - val_accuracy: 0.4668
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5574 - accuracy: 0.7277 - val_loss: 19727.1133 - val_accuracy: 0.4668
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5535 - accuracy: 0.7302 - val_loss: 47891.6250 - val_accuracy: 0.5332
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5518 - accuracy: 0.7318 - val_loss: 3662.1860 - val_accuracy: 0.4668
Epoch 5/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5508 - accuracy: 0.7304 - val_loss: 3430.9932 - val_accuracy: 0.4668
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5494 - accuracy: 0.7320 - val_loss: 6536.4272 - val_accuracy: 0.4668
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5493 - accuracy: 0.7339 - val_loss: 87381.531

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

nn.save("AlphabetSoupCharity_Optimization.h5")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7236 - 630ms/epoch - 2ms/step
Loss: 0.5545100569725037, Accuracy: 0.7236151695251465


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
# Try adding a third hidden layer
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu', input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 40)                1760      
                                                                 
 dense_7 (Dense)             (None, 10)                410       
                                                                 
 dense_8 (Dense)             (None, 2)                 22        
                                                                 
 dense_9 (Dense)             (None, 1)                 3         
                                                                 
Total params: 2195 (8.57 KB)
Trainable params: 2195 (8.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5838 - accuracy: 0.7034
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5527 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7271
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5497 - accuracy: 0.7306
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7340
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accura

In [49]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5518 - accuracy: 0.7271 - 551ms/epoch - 2ms/step
Loss: 0.5518204569816589, Accuracy: 0.7271137237548828


In [51]:
###########################
# Reintroduce the NAME column to see if this will improve model performance.
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_application_df = application_df.drop(columns=['EIN'])

# use the variable name `application_types_to_replace`
application_types_to_replace = clean_application_df['APPLICATION_TYPE'].value_counts()[clean_application_df['APPLICATION_TYPE'].value_counts() < 500]
# Replace in dataframe
for app in application_types_to_replace.index:
    clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")

# use the variable name `classifications_to_replace`
classifications_to_replace =  clean_application_df['CLASSIFICATION'].value_counts()[clean_application_df['CLASSIFICATION'].value_counts() < 1000]
# Replace in dataframe
for cls in classifications_to_replace.index:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other")

classifications_to_replace =  clean_application_df['NAME'].value_counts()[clean_application_df['NAME'].value_counts() < 5]

# Replace in dataframe
for cls in classifications_to_replace.index:
    clean_application_df['NAME'] = clean_application_df['NAME'].replace(cls,"Other")

# Check to make sure binning was successful
clean_application_df['NAME'].value_counts()

Other                                                                19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: NAME, Length: 403, dtype: int64

In [53]:
df_wdummies = pd.get_dummies(clean_application_df)
df_wdummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
# Split our preprocessed data into our features and target arrays
y = df_wdummies.IS_SUCCESSFUL.values
X = df_wdummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [55]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [56]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu', input_dim=446))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 40)                17880     
                                                                 
 dense_11 (Dense)            (None, 10)                410       
                                                                 
 dense_12 (Dense)            (None, 2)                 22        
                                                                 
 dense_13 (Dense)            (None, 1)                 3         
                                                                 
Total params: 18315 (71.54 KB)
Trainable params: 18315 (71.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [58]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5168 - accuracy: 0.7530
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4540 - accuracy: 0.7714
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4485 - accuracy: 0.7742
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4461 - accuracy: 0.7753
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4446 - accuracy: 0.7753
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4422 - accuracy: 0.7851
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4403 - accuracy: 0.7903
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4390 - accuracy: 0.7906
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4379 - accuracy: 0.7930
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4368 - accura

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4593 - accuracy: 0.7883 - 438ms/epoch - 2ms/step
Loss: 0.4593333601951599, Accuracy: 0.7883381843566895


In [ ]:
#####
# Try eliminating uninformative columns to improve the accuracy score
print(clean_application_df.columns)
print(application_df_wdummies.columns)

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')
Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative'

In [60]:
df_noAffiliation = clean_application_df.drop(columns='AFFILIATION')
df_noAffiliation.head()

,NAME,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,C1000,ProductDev,Association,1,0,N,5000,1
1,Other,T3,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,Other,T5,C3000,ProductDev,Association,1,0,N,5000,0
3,Other,T3,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Other,T3,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [61]:
df_noAffiliation_wdummies = pd.get_dummies(df_noAffiliation)
df_noAffiliation_wdummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [62]:
# Split our preprocessed data into our features and target arrays
y = df_noAffiliation_wdummies.IS_SUCCESSFUL.values
X = df_noAffiliation_wdummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [63]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu', input_dim=440))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 40)                17640     
                                                                 
 dense_15 (Dense)            (None, 10)                410       
                                                                 
 dense_16 (Dense)            (None, 2)                 22        
                                                                 
 dense_17 (Dense)            (None, 1)                 3         
                                                                 
Total params: 18075 (70.61 KB)
Trainable params: 18075 (70.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [66]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5961 - accuracy: 0.6841
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7196
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5350 - accuracy: 0.7282
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5290 - accuracy: 0.7343
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5251 - accuracy: 0.7375
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5197 - accuracy: 0.7419
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5119 - accuracy: 0.7490
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5024 - accuracy: 0.7531
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5004 - accuracy: 0.7538
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4952 - accura

In [67]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4532 - accuracy: 0.7887 - 546ms/epoch - 2ms/step
Loss: 0.45316004753112793, Accuracy: 0.788688063621521
